<a href="https://colab.research.google.com/github/subhashbs36/DeepLearning_Colab/blob/main/Subhash_mnist_neural_net1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#hide
! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.8/719.8 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 10.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
Mounted at /content/gdrive


In [2]:
#hide
from fastai.vision.all import *
from fastbook import *

matplotlib.rc('image', cmap='Greys')

In [3]:
path = untar_data(URLs.MNIST_SAMPLE)


In [64]:
#hide
Path.BASE_PATH = path

In [65]:
threes = (path/'train'/'3').ls().sorted()
sevens = (path/'train'/'7').ls().sorted()
threes

(#6131) [Path('train/3/10.png'),Path('train/3/10000.png'),Path('train/3/10011.png'),Path('train/3/10031.png'),Path('train/3/10034.png'),Path('train/3/10042.png'),Path('train/3/10052.png'),Path('train/3/1007.png'),Path('train/3/10074.png'),Path('train/3/10091.png')...]

In [66]:
valid_threes = (path/'valid'/'3').ls().sorted()
valid_sevens = (path/'valid'/'7').ls().sorted()
valid_threes

(#1010) [Path('valid/3/1020.png'),Path('valid/3/1028.png'),Path('valid/3/1042.png'),Path('valid/3/1062.png'),Path('valid/3/1066.png'),Path('valid/3/1067.png'),Path('valid/3/1069.png'),Path('valid/3/1072.png'),Path('valid/3/1092.png'),Path('valid/3/1095.png')...]

In [89]:
stacked_threes = torch.stack([tensor(Image.open(i)) for i in threes]).float()/255
stacked_sevens = torch.stack([tensor(Image.open(i)) for i in sevens]).float()/255

In [90]:
stacked_valid_threes = torch.stack([tensor(Image.open(i)) for i in valid_threes]).float()/255
stacked_valid_sevens = torch.stack([tensor(Image.open(i)) for i in valid_sevens]).float()/255

In [91]:
x_train = torch.cat([stacked_threes, stacked_sevens]).view(-1, 28*28)
y_train = tensor([1]*len(threes) + [0]*len(sevens)).unsqueeze(1)
x_train.shape, y_train.shape

(torch.Size([12396, 784]), torch.Size([12396, 1]))

In [92]:
x_valid = torch.cat([stacked_valid_threes, stacked_valid_sevens]).view(-1, 28*28)
y_valid = tensor([1]*len(valid_threes) + [0]*len(valid_sevens)).unsqueeze(1)
x_valid.shape, y_valid.shape

(torch.Size([2038, 784]), torch.Size([2038, 1]))

In [93]:
dset = list(zip(x_train, y_train))
valid_dset = list(zip(x_valid, y_valid))

dl = DataLoader(dset, batch_size=256)
valid_dl = DataLoader(valid_dset, batch_size=256)

# **Building A Optimizer from Scratch Core**

In [94]:
linear_model = nn.Linear(28*28, 1)
lr = 0.01

In [95]:
def mnist_loss(predicted, target):
  predicted = predicted.sigmoid()
  return torch.where(target==1, 1-predicted, predicted).mean()

In [96]:
class basicOpti():
  def __init__(self, params, lr):
    self.params = list(params)
    self.lr = lr

  def step(self):
    for p in self.params:
      p.data -= p.grad * self.lr

  def zero_grad(self):
    for p in self.params:
      p.grad.zero_()

In [97]:
def calc_grad(xb, yb, model):
  pred = model(xb)
  loss = mnist_loss(pred, yb)
  loss.backward()

In [98]:
def batch_accuracy(xa, ya):
  predicted = xa.sigmoid()
  correct = (predicted>0.5) == ya
  return correct.float().mean()

In [99]:
optimizer = basicOpti(linear_model.parameters(), lr)

In [100]:
def train_epoch(model):
  for xa, ya in dl:
    calc_grad(xa, ya, model)
    optimizer.step()
    optimizer.zero_grad()

In [101]:
def validate_epoch(model):
  accuracy = [batch_accuracy(model(xa), ya) for xa, ya in valid_dl]
  return round(torch.stack(accuracy).mean().item(), 4)

In [102]:
def train_model(model, epochs):
  for epoch in range(epochs):
    train_epoch(model)
    print(validate_epoch(model), end=' ')

In [103]:
train_model(linear_model, 20)

0.9057 0.9209 0.9385 0.9521 0.957 0.9604 0.9609 0.9619 0.9614 0.9623 0.9623 0.9623 0.9638 0.9633 0.9633 0.9643 0.9648 0.9653 0.9653 0.9658 

[link text](https://)fastai provides the `SGD` class which, by default, does the same thing as our `basicOpti`:


In [104]:
linear_model = nn.Linear(28*28,1)
optimizer = SGD(linear_model.parameters(), lr)
train_model(linear_model, 20)

0.9379 0.9268 0.9443 0.9546 0.958 0.9604 0.9624 0.9623 0.9619 0.9628 0.9628 0.9633 0.9638 0.9648 0.9643 0.9643 0.9653 0.9657 0.9657 0.9657 

# **Now using FastAI**

fastai also provides `Learner.fit`, which we can use instead of `train_model`. To create a `Learner` we first need to create a **`DataLoaders`**, by passing in our training and validation **`DataLoader`**s:

In [111]:
dataset = DataLoaders(dl, valid_dl) #Dataloader in Dataloaders

In [112]:
learn = Learner(dataset, nn.Linear(28*28, 1), loss_func=mnist_loss, opt_func=SGD, metrics=batch_accuracy)

In [114]:
learn.fit(30, lr=0.001)

epoch,train_loss,valid_loss,batch_accuracy,time
0,0.093161,0.095724,0.962709,00:00
1,0.092875,0.095107,0.963690,00:00
2,0.092553,0.094521,0.963690,00:00
3,0.092210,0.093963,0.963690,00:00
4,0.091858,0.093428,0.964671,00:00
5,0.091504,0.092915,0.965162,00:00
6,0.091153,0.092422,0.965653,00:00
7,0.090805,0.091946,0.965162,00:00
8,0.090462,0.091486,0.965162,00:00
9,0.090122,0.091040,0.965653,00:00


# **Building a Simple NN using FastAI**

Random Initializer with Grad for parameter initializing

In [115]:
def init_params(size, std=1.0):
  return (torch.randn(size)*std).requires_grad_()

## Adding a Nonlinearity

So far we have a general procedure for optimizing the parameters of a function, and we have tried it out on a very boring function: a simple linear classifier. A linear classifier is very constrained in terms of what it can do. To make it a bit more complex (and able to handle more tasks), we need to add something nonlinear between two linear classifiers—this is what gives us a neural network.

Here is the entire definition of a basic neural network:

In [116]:
def simple_net(xb):
    res = xb@w1 + b1
    res = res.max(tensor(0.0)) #or F.relu(res)
    res = res@w2 + b2
    return res

In [117]:
w1 = init_params((28*28,30))
b1 = init_params(30)
w2 = init_params((30,1))
b2 = init_params(1)

The key point about this is that `w1` has 30 output activations (which means that `w2` must have 30 input activations, so they match). That means that the first layer can construct 30 different features, each representing some different mix of pixels. You can change that `30` to anything you like, to make the model more or less complex.

That little function `res.max(tensor(0.0))` is called a *rectified linear unit*, also known as *ReLU*. We think we can all agree that *rectified linear unit* sounds pretty fancy and complicated... But actually, there's nothing more to it than `res.max(tensor(0.0))`—in other words, replace every negative number with a zero. This tiny function is also available in PyTorch as `F.relu`:

# Using PyTorch Build Simple NN

In [121]:
simple_net = nn.Sequential(
    nn.Linear(28*28, 30),
    nn.ReLU(),
    nn.Linear(30, 1)
)

`nn.Sequential` creates a module that will call each of the listed layers or functions in turn.

`nn.ReLU` is a PyTorch module that does exactly the same thing as the `F.relu` function. Most functions that can appear in a model also have identical forms that are modules. Generally, it's just a case of replacing `F` with `nn` and changing the capitalization. When using `nn.Sequential`, PyTorch requires us to use the module version. Since modules are classes, we have to instantiate them, which is why you see `nn.ReLU()` in this example.

Because `nn.Sequential` is a module, we can get its parameters, which will return a list of all the parameters of all the modules it contains. Let's try it out! As this is a deeper model, we'll use a lower learning rate and a few more epochs.

In [122]:
learn = Learner(dataset, simple_net, loss_func=mnist_loss, opt_func=SGD, metrics=batch_accuracy)

In [123]:
#hide_output
learn.fit(40, 0.1)

epoch,train_loss,valid_loss,batch_accuracy,time
0,0.312166,0.404465,0.506379,00:00
1,0.145596,0.231331,0.802257,00:00
2,0.080657,0.115488,0.915604,00:00
3,0.052939,0.077457,0.943081,00:00
4,0.040113,0.060312,0.955348,00:00
5,0.033582,0.050774,0.965653,00:00
6,0.029834,0.044804,0.966143,00:00
7,0.027405,0.040724,0.968106,00:00
8,0.025659,0.037764,0.970069,00:00
9,0.024312,0.035506,0.971050,00:00
